<a href="https://colab.research.google.com/github/Wenjie0o0/ClassAI/blob/main/Resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
num_epochs = 10
batch_size = 64
learning_rate = 1e-4 #0.0001

In [4]:
transform = transforms.Compose(
    [ transforms.Resize([224,224]),
      transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
class Resnet(nn.Module):
  def __init__(self,block,in_channels=64,num_classes=10):
    super().__init__()
    self.in_channels=in_channels
    
    self.conv1=conv_block(in_channels=3,out_channels=64,kernel_size=7,stride=2,padding=3)
    self.relu=nn.ReLU()
    self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)

    self.layer1=self.make_layer(block,64,3,stride=1)
    self.layer2=self.make_layer(block,128,4,stride=2)
    self.layer3=self.make_layer(block,256,6,stride=2)
    self.layer4=self.make_layer(block,512,3,stride=2)

    self.avgpool=nn.AdaptiveAvgPool2d((1,1))
    self.fc=nn.Linear(2048,num_classes)

  def forward(self,x):
    x=self.conv1(x)
    x=self.relu(x)
    x=self.maxpool(x)

    x=self.layer1(x)
    x=self.layer2(x)
    x=self.layer3(x)
    x=self.layer4(x)

    x=self.avgpool(x)
    x=x.reshape(x.shape[0],-1)
    x=self.fc(x)
    return x

  def make_layer(self,block,in_channels,block_num,stride):
    block_list=[]
    downsample=None
    if(stride!=1 or self.in_channels!=in_channels*4):
      downsample=nn.Sequential(
          conv_block(self.in_channels,in_channels*4,stride=stride,kernel_size=1)
      )
      
      block_list.append(block(self.in_channels,in_channels,stride=stride,downsample=downsample))
      self.in_channels=in_channels*4

      for i in range(block_num-1):
        block_list.append(block(self.in_channels,in_channels))
      return nn.Sequential(*block_list)


class conv_block(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super().__init__()
        
        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels)
        
    def forward(self, x):
        return self.batchnorm(self.conv(x))

class Bottleneck(nn.Module):
  def __init__(self,in_channels,out_channels,stride=1,downsample=None):
    super(Bottleneck,self).__init__()
    self.conv1=conv_block(in_channels,out_channels,kernel_size=1,stride=1,padding=0)
    self.conv2=conv_block(out_channels,out_channels,kernel_size=3,stride=stride,padding=1)
    self.conv3=conv_block(out_channels,out_channels*4,kernel_size=1,stride=1,padding=0)
    self.relu=nn.ReLU()

    self.downsample=downsample
    self.stride=stride

  def forward(self,x):
    residual=x.clone()
    x=self.conv1(x)
    x=self.relu(x)
    x=self.conv2(x)
    x=self.relu(x)
    x=self.conv3(x)
    

    if self.downsample is not None:
      residual=self.downsample(residual)
    x+=residual
    x=self.relu(x)

    return x

In [6]:
resnet = Resnet(Bottleneck).to(device)

In [7]:
print(resnet)

Resnet(
  (conv1): conv_block(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): conv_block(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv2): conv_block(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv3): conv_block(
        (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (batchnorm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReL

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.parameters(), lr=learning_rate)

In [9]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(resnet)

23555082

In [ ]:
n_total_steps = len(train_loader)
for epoch in tqdm(range(num_epochs)):
  for i, (images, labels) in enumerate(train_loader):
    # origin shape: [64, 3, 224, 224]
    images = images.to(device)
    labels = labels.to(device)
    # Forward pass
    
    outputs = resnet(images)
    loss = criterion(outputs, labels)
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch [1/10], Step [100/782], Loss: 1.6446
Epoch [1/10], Step [200/782], Loss: 1.7051
Epoch [1/10], Step [300/782], Loss: 1.3484
Epoch [1/10], Step [400/782], Loss: 1.3323
Epoch [1/10], Step [500/782], Loss: 1.3191
Epoch [1/10], Step [600/782], Loss: 1.4661
Epoch [1/10], Step [700/782], Loss: 1.1586


 10%|█         | 1/10 [10:07<1:31:07, 607.48s/it]

Epoch [2/10], Step [100/782], Loss: 1.2545
Epoch [2/10], Step [200/782], Loss: 1.0592
Epoch [2/10], Step [300/782], Loss: 0.7239
Epoch [2/10], Step [400/782], Loss: 0.9873
Epoch [2/10], Step [500/782], Loss: 0.7252
Epoch [2/10], Step [600/782], Loss: 0.8789
Epoch [2/10], Step [700/782], Loss: 0.8642


 20%|██        | 2/10 [20:12<1:20:50, 606.32s/it]

Epoch [3/10], Step [100/782], Loss: 0.6957
Epoch [3/10], Step [200/782], Loss: 0.7859


In [ ]:
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  n_class_correct = [0 for i in range(10)]
  n_class_samples = [0 for i in range(10)]
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = resnet(images)
    # max returns (value ,index)
    _, predicted = torch.max(outputs, 1)
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()

  acc = 100.0 * n_correct / n_samples
  print(f'Accuracy of the network: {acc} %')